In [1]:
# %pip install coniferest

In [2]:
from pathlib import Path

import pandas as pd
from coniferest.label import Label
from coniferest.pineforest import PineForest
from coniferest.session import Session
from coniferest.session.callback import TerminateAfter, prompt_decision_callback

In [3]:
features_root = Path("object_lc_features")
data = pd.read_parquet(features_root, partitioning=None)
data = data.query(
    "r_psfMag > 21"
    f" and {' + '.join(f'observation_count_flux_det_{b}_direct' for b in 'ugrizy')} >= 8"
)
display(data)
ids = data.objectId.to_numpy()
X = data.drop(columns=["objectId"]).to_numpy()

,objectId,u_psfMag,g_psfMag,r_psfMag,i_psfMag,z_psfMag,y_psfMag,lg_anderson_darling_normal_flux_det_u_direct,duration_flux_det_u_direct,observation_count_flux_det_u_direct,...,duration_src_y_dia,observation_count_src_y_dia,villar_fit_mag_full_amplitude_src_y_dia,villar_fit_baseline_amplitude_ratio_src_y_dia,villar_fit_rise_time_src_y_dia,villar_fit_fall_time_src_y_dia,villar_fit_plateau_rel_amplitude_src_y_dia,villar_fit_plateau_duration_src_y_dia,ln1p_villar_fit_reduced_chi2_src_y_dia,_healpix_29
947,4601777219637871169,30.000000,24.197224,23.402853,22.503044,22.309006,30.000000,-1.0,-1.0,0.0,...,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1258603637639219331
948,4601777219637877071,30.000000,22.595806,21.375578,20.966553,20.819817,30.000000,-1.0,-1.0,0.0,...,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1258603637763774031
993,4601777219637871293,30.000000,23.522381,23.011576,22.234219,22.112637,30.000000,-1.0,-1.0,0.0,...,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1258603671608386395
1136,4601777219637877459,30.000000,24.047750,22.641720,21.779213,21.516882,30.000000,-1.0,-1.0,0.0,...,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1258603773688277856
1154,4601777219637871363,30.000000,30.000000,22.622952,22.275824,22.297962,30.000000,-1.0,-1.0,0.0,...,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1258603789794624383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993558,2226928860664971236,24.279108,24.740477,24.481028,23.915998,23.611267,24.150534,-1.0,-1.0,0.0,...,1.192505,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2528753590453949347
993559,2226928860664971235,25.424473,24.452978,23.833771,23.341406,22.700319,22.463230,-1.0,-1.0,0.0,...,1.192505,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2528753591229728565
993560,2226928860664971478,25.461327,24.008961,22.432995,20.229874,19.132221,18.512836,-1.0,-1.0,0.0,...,1.192505,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2528753592033115693
993579,2226928860664971816,24.921890,24.560822,24.598227,24.614429,24.277746,24.228151,-1.0,-1.0,0.0,...,1.192505,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2528753594269858397


In [4]:
# import sys

# from lsst.daf.butler import Butler

# sys.path.append("/sdf/data/rubin/user/kostya/linccf/internal/single_object")
# from single_object import make_figure


# repo = "/repo/main"
# instrument = "LSSTComCam"
# release = "v29_0_0_rc5"
# collection = f"LSSTComCam/runs/DRP/DP1/{release}/DM-49865"

# butler = Butler(repo, collections=collection)
# hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats") / release

# def decision_callback(metadata, data, session) -> Label:
#     make_figure(metadata, butler, hats_path, image_type="direct")
#     # return prompt_decision_callback(metadata, data, session)
#     return Label.REGULAR

In [5]:
session = Session(
    data=X,
    metadata=ids,
    model=PineForest(random_seed=0, n_subsamples=4098, n_trees=1024),
    decision_callback=prompt_decision_callback,
    # on_decision_callbacks=TerminateAfter(20),
)
session.run()

Is 2226783725130104506 anomaly? [y/N]:

  n


Is 2226796919269634157 anomaly? [y/N]:

  n


Is 2226796919269634158 anomaly? [y/N]:

...

Is 2429925095431484897 anomaly? [y/N]:

  n


Is 199385438580901728 anomaly? [y/N]:

  n


Is 199429419046020026 anomaly? [y/N]: